##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Penguin テンプレートで独自データ用の TFX パイプラインを作成する

---


注意: このチュートリアルは、Google Cloud [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench) で実行することをお勧めします。[Vertex AI Workbench に移動](https://console.cloud.google.com/vertex-ai/workbench)してください。

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_template"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/tfx/penguin_template.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a>
</td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/tfx/penguin_template.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHubでソースを表示</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tfx/tutorials/tfx/penguin_template.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table></div>

## 重み値のみを保存。（通常、モデルのトレーニング時に使用）。

このドキュメントでは、TFX Python パッケージに含まれる *penguin template* を使用して、独自のデータセットの TensorFlow Extended（TFX）パイプラインを作成する方法を説明します。作成されるパイプラインでは最初に [Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/articles/intro.html) データセットが使用されますが、独自のデータセットに合わせてこのパイプラインを変換します。


### 前提条件

- Linux / MacOS
- Python 3.6-3.8
- Jupyter notebook


## ステップ 1. 事前定義さあれたテンプレートをプロジェクトディレクトリにコピーする

このステップでは、作業パイプラインプロジェクトディレクトリを作成し、TFX の *penguin テンプレート*からファイルをコピーします。これは、独自の TFX パイプラインプロジェクトの足場として捉えるとよいでしょう。

### Pip を更新する

Colab で実行している場合は、最新バージョンの Pip を使用していることを確認してください。ローカルシステムは、もちろん個別に更新できます。

注意: Vertex AI Workbench で実行している場合でも、おそらく更新することが推奨されます。

In [ ]:
import sys
if 'google.colab' in sys.modules:
  !pip install --upgrade pip

### 必要なパッケージをインストールする

まず、TFX と TensorFlow Model Analysis（TFMA）をインストールします。


In [ ]:
!pip install -U tfx tensorflow-model-analysis

TFX のバージョンを確認します。

In [ ]:
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tfx

print('TF version: {}'.format(tf.__version__))
print('TFMA version: {}'.format(tfma.__version__))
print('TFX version: {}'.format(tfx.__version__))

以上でパイプラインを作成する準備ができました。

`PROJECT_DIR` を自分の環境の適切な場所に設定します。デフォルト値は `~/imported/${PIPELINE_NAME}` で、[Google Cloud AI Platform Notebook](https://console.cloud.google.com/ai-platform/notebooks/) 環境の場合に最適です。

以下の `PIPELINE_NAME` を変更することで、パイプラインに別の名前を付けることができます。また、これはファイルが配置されるプロジェクトディレクトリ名にもなります。


In [ ]:
PIPELINE_NAME="my_pipeline"
import os
# Set this project directory to your new tfx pipeline project.
PROJECT_DIR=os.path.join(os.path.expanduser("~"), "imported", PIPELINE_NAME)

### テンプレートファイルをコピーする

TFX には、TFX Python パッケージとともに `penguin` テンプレートが含まれます。`penguin` テンプレートには、データセットをパイプラインに取り込む指示が多数含まれています。このチュートリアルでは、それを目的としています。

`tfx template copy` CLI コマンドは、事前定義されたテンプレートファイルをプロジェクトディレクトリにコピーします。

In [ ]:
# Set `PATH` to include user python binary directory and a directory containing `skaffold`.
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

!tfx template copy \
  --pipeline-name={PIPELINE_NAME} \
  --destination-path={PROJECT_DIR} \
  --model=penguin

このノートブックの作業ディレクトリコンテキストをプロジェクトディレクトリに変更します。

In [ ]:
%cd {PROJECT_DIR}

> 注意: JupyterLab または Google Cloud AI Platform Notebook を使用している場合は、作成されたプロジェクトディレクトリをクリックして、左側の `File Browser` のディレクトリを忘れずに変更してください。

### コピーしたソースファイルを閲覧する

TFX テンプレートは、Python ソースコードやサンプルデータなど、パイプラインを構築するための基本的なスキャフォールドファイルを提供しています。`penguin` テンプレートは、[ペンギンの例](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin)と同じ *Palmer Penguins* データセットを使用しています。

ここでは、各 Python ファイルを簡単に紹介します。

- `pipeline`- このディレクトリには、パイプラインの定義が含まれています。
    - `configs.py` — パイプライン ランナーの共通定数を定義します。
    - `pipeline.py` — TFX コンポーネントとパイプラインを定義します。
- `models` - このディレクトリには ML モデルの定義が含まれています。
    - `features.py`、`features_test.py` — モデルの特徴を定義します。
    - `preprocessing.py`、`preprocessing_test.py` — データの前処理ルーチンを定義します。
    - `constants.py` — モデルの定数を定義します。
    - `model.py`、`model_test.py` — TensorFlow などの ML フレームワークを使って ML モデルを定義します。
- `local_runner.py` — ローカルオーケストレーションエンジンを使用するローカル環境のランナーを定義します。
- `kubeflow_runner.py` — Kubeflow Pipelines オーケストレーションエンジン用のランナーを定義します。


デフォルトでは、テンプレートには標準の TFX コンポーネントしか含まれていません。アクションをカスタマイズする必要がある場合は、パイプライン用のカスタムコンポーネントを作成できます。詳細については、[TFX カスタムコンポーネントガイド](https://www.tensorflow.org/tfx/guide/understanding_custom_components)をご覧ください。

#### Unit-test ファイル

名前に `_test.py` が含まれているファイルがあることに気が付きましたか。これらはパイプラインの単体テストであり、独自のパイプラインを実装するときに単体テストを追加することをお勧めします。テストファイルのモジュール名に `-m` フラグを指定すると、単体テストを実行できます。通常、モジュール名を取得するには、`.py` 拡張子を削除し、`/` を `.` に置き換えます。以下に例を示します。

In [ ]:
import sys
!{sys.executable} -m models.features_test

### ローカル環境に TFX パイプラインを作成する

TFX は、パイプラインを実行するオーケストレーションエンジンを複数サポートしています。ここでは、ローカルオーケストレーションエンジンを使用します。ローカルオーケストレーションエンジンは、その他の依存関係を使用せずに実行し、リモート計算クラスタではなくローカル環境で実行するため、開発やデバッグに適しています。

`local_runner.py` を使用して、ローカルオーケストレータでパイプラインを実行します。実行する前に、パイプラインを作成する必要があります。パイプラインは、`pipeline create` コマンドで作成できます。


In [ ]:
!tfx pipeline create --engine=local --pipeline_path=local_runner.py

`pipeline create` コマンドは、実際に実行せずに`local_runner.py` に定義されたパイプラインを登録します。

次の手順では、作成したパイプラインを `run create` コマンドで実行します。


## ステップ 2. 独自のデータをパイプラインに取り込む

最初のパイプラインでは、テンプレートに含まれるペンギンのデータセットを取り込みます。データをパイプラインに取り込む必要があり、ほとんどの TFX パイプラインは ExampleGen コンポーネントから始まります。

### ExampleGen を選択する

データは、ローカルまたは分散ファイルシステム、またはクエリ可能なシステムなど、パイプラインがアクセスする場所であれば、どこにでも格納できます。TFX には、データを TFX パイプラインに取り込むための [`ExampleGen` コンポーネント](https://www.tensorflow.org/tfx/guide/examplegen)が複数用意されています。以下のサンプル生成コンポーネントのいずれかを選択できます。

- CsvExampleGen: ディレクトリの CSV ファイルを読み取ります。[ペンギンの例](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin)と[シカゴタクシーの例](https://github.com/tensorflow/tfx/tree/master/tfx/examples/chicago_taxi_pipeline)で使用されています。
- ImportExampleGen: TFRecord ファイルを TF Example データフォーマットで取ります。[MNIST の例](https://github.com/tensorflow/tfx/tree/master/tfx/examples/mnist)で使用されています。
- FileBasedExampleGen: [Avro](https://github.com/tensorflow/tfx/blob/master/tfx/components/example_gen/custom_executors/avro_executor.py) または [Parquet](https://github.com/tensorflow/tfx/blob/master/tfx/components/example_gen/custom_executors/parquet_executor.py) フォーマット用。
- [BigQueryExampleGen](https://www.tensorflow.org/tfx/api_docs/python/tfx/extensions/google_cloud_big_query/example_gen/component/BigQueryExampleGen): Google Cloud BigQuery にデータを直接読み取ります。[シカゴタクシーの例](https://github.com/tensorflow/tfx/tree/master/tfx/examples/chicago_taxi_pipeline)で使用されています。

また、独自の ExampleGen を作成することも可能です。たとえば、tfx にはデータソースとして [Presto を使用するカスタム ExecampleGen](https://github.com/tensorflow/tfx/tree/master/tfx/examples/custom_components/presto_example_gen) が含まれています。カスタム Executor の使用と開発方法についての詳細は、[ガイド](https://www.tensorflow.org/tfx/guide/examplegen#custom_examplegen)をご覧ください。

使用する ExampleGen を決定したら、データを使用するためのパイプライン定義を変更する必要があります。

1. `local_runner.py` の `DATA_PATH` を変更して、ファイルの場所に設定します。

- ローカル環境にファイルがある場合は、そのパスを指定します。パイプラインの開発またはデバッグには、このオプションが最適です。
- GCS にファイルがある場合は、`gs://{bucket_name}/...` からのパスを使用できます。[`gsutil`](https://cloud.google.com/storage/docs/gsutil) などを使って、ターミナルから GCS にアクセスできることを確認してください。必要であれば、[Google Cloud の認証ガイド](https://cloud.google.com/sdk/docs/authorizing)をご覧ください。
- BigQueryExampleGen などのクエリベースの ExampleGen を使用する場合は、データソースからデータを選択するためのクエリ文が必要です。Google Cloud BigQuery をデータソースとして使用する場合には、このほかにもいくつか設定が必要です。
    - `pipeline/configs.py`:
        - `GOOGLE_CLOUD_PROJECT` と `GCS_BUCKET_NAME` を自分の GCP プロジェクトとバケット名に変更します。このバケットは、パイプラインを実行する前に存在する必要があります。
        - `BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS` 変数をコメント解除します。
        - `BIG_QUERY_QUERY` 変数をコメント解除し、**自分のクエリ文**に設定します。
    - `local_runner.py`:
        - `pipeline.create_pipeline()` の `data_path` 引数をコメントアウトし、代わりに `query` 引数をコメント解除します。
    - `pipeline/pipeline.py`:
        - `create_pipeline()` の `data_path` 引数をコメントアウトし、`query` 引数をコメント解除します。
        - CsvExampleGen の代わりに [BigQueryExampleGen](https://www.tensorflow.org/tfx/api_docs/python/tfx/extensions/google_cloud_big_query/example_gen/component/BigQueryExampleGen) を使用します。

1. `pipeline/pipeline.py` で、既存の CsvExampleGen を自分の ExampleGen クラスに置き換えます。それぞれの ExampleGen クラスのシグネチャは異なります。詳細については、[ExampleGen コンポーネントガイド](https://www.tensorflow.org/tfx/guide/examplegen)をご覧ください。`pipeline/pipeline.py` に `import` 文を使って、必要なモジュールを忘れずにインポートしてください。

最初のパイプラインは、`ExampleGen`、`StatisticsGen`、`SchemaGen`、`ExampleValidator` の 4 つのコンポーネントで構成されています。`StatisticsGen`、`SchemaGen`、および `ExampleValidator` については、何の変更も必要ありません。このパイプラインを初めて実行してみましょう。

In [ ]:
# Update and run the pipeline.
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

パイプラインが正常に完了すると、「Component ExampleValidator is finished.」が表示されます。

### パイプラインの出力を調べる

TFX パイプラインは、アーティファクトと、アーティファクトとパイプラインの実行のメタデータを含む [metadata DB(MLMD)](https://www.tensorflow.org/tfx/guide/mlmd) の 2 種類の出力を生成します。出力の場所は、`local_runner.py` で定義されいます。デフォルトでは、アーティファクトは `tfx_pipeline_output` ディレクトリに、メタデータは sqlite データベースとして `tfx_metadata` ディレクトリに保存されています。

これらの出力は、MLMD API を使って調べられます。まず、生成されたばかりの出力アーティファクトを検索するユーティリティ関数を定義しましょう。

In [ ]:
import tensorflow as tf
import tfx
from ml_metadata import errors
from ml_metadata.proto import metadata_store_pb2
from tfx.types import artifact_utils

# TODO(b/171447278): Move these functions into TFX library.

def get_latest_executions(store, pipeline_name, component_id = None):
  """Fetch all pipeline runs."""
  if component_id is None:  # Find entire pipeline runs.
    run_contexts = [
        c for c in store.get_contexts_by_type('run')
        if c.properties['pipeline_name'].string_value == pipeline_name
    ]
  else:  # Find specific component runs.
    run_contexts = [
        c for c in store.get_contexts_by_type('component_run')
        if c.properties['pipeline_name'].string_value == pipeline_name and
           c.properties['component_id'].string_value == component_id
    ]
  if not run_contexts:
    return []
  # Pick the latest run context.
  latest_context = max(run_contexts,
                       key=lambda c: c.last_update_time_since_epoch)
  return store.get_executions_by_context(latest_context.id)

def get_latest_artifacts(store, pipeline_name, component_id = None):
  """Fetch all artifacts from latest pipeline execution."""
  executions = get_latest_executions(store, pipeline_name, component_id)

  # Fetch all artifacts produced from the given executions.
  execution_ids = [e.id for e in executions]
  events = store.get_events_by_execution_ids(execution_ids)
  artifact_ids = [
      event.artifact_id for event in events
      if event.type == metadata_store_pb2.Event.OUTPUT
  ]
  return store.get_artifacts_by_id(artifact_ids)

def find_latest_artifacts_by_type(store, artifacts, artifact_type):
  """Get the latest artifacts of a specified type."""
  # Get type information from MLMD
  try:
    artifact_type = store.get_artifact_type(artifact_type)
  except errors.NotFoundError:
    return []
  # Filter artifacts with type.
  filtered_artifacts = [aritfact for aritfact in artifacts
                        if aritfact.type_id == artifact_type.id]
  # Convert MLMD artifact data into TFX Artifact instances.
  return [artifact_utils.deserialize_artifact(artifact_type, artifact)
      for artifact in filtered_artifacts]


from tfx.orchestration.experimental.interactive import visualizations

def visualize_artifacts(artifacts):
  """Visualizes artifacts using standard visualization modules."""
  for artifact in artifacts:
    visualization = visualizations.get_registry().get_visualization(
        artifact.type_name)
    if visualization:
      visualization.display(artifact)

from tfx.orchestration.experimental.interactive import standard_visualizations
standard_visualizations.register_standard_visualizations()

import pprint

from tfx.orchestration import metadata
from tfx.types import artifact_utils
from tfx.types import standard_artifacts

def preview_examples(artifacts):
  """Preview a few records from Examples artifacts."""
  pp = pprint.PrettyPrinter()
  for artifact in artifacts:
    print("==== Examples artifact:{}({})".format(artifact.name, artifact.uri))
    for split in artifact_utils.decode_split_names(artifact.split_names):
      print("==== Reading from split:{}".format(split))
      split_uri = artifact_utils.get_split_uri([artifact], split)

      # Get the list of files in this directory (all compressed TFRecord files)
      tfrecord_filenames = [os.path.join(split_uri, name)
                            for name in os.listdir(split_uri)]
      # Create a `TFRecordDataset` to read these files
      dataset = tf.data.TFRecordDataset(tfrecord_filenames,
                                        compression_type="GZIP")
      # Iterate over the first 2 records and decode them.
      for tfrecord in dataset.take(2):
        serialized_example = tfrecord.numpy()
        example = tf.train.Example()
        example.ParseFromString(serialized_example)
        pp.pprint(example)

import local_runner

metadata_connection_config = metadata.sqlite_metadata_connection_config(
              local_runner.METADATA_PATH)

次に、MLMD から出力アーティファクトのメタデータを読み取ります。

In [ ]:
with metadata.Metadata(metadata_connection_config) as metadata_handler:
    # Search all aritfacts from the previous pipeline run.
    artifacts = get_latest_artifacts(metadata_handler.store, PIPELINE_NAME)
    # Find artifacts of Examples type.
    examples_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.Examples.TYPE_NAME)
    # Find artifacts generated from StatisticsGen.
    stats_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.ExampleStatistics.TYPE_NAME)
    # Find artifacts generated from SchemaGen.
    schema_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.Schema.TYPE_NAME)
    # Find artifacts generated from ExampleValidator.
    anomalies_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.ExampleAnomalies.TYPE_NAME)

これで、各コンポーネントの出力を調べられるようになりました。[Tensorflow Data Validation(TFDV)](https://www.tensorflow.org/tfx/data_validation/get_started) が `StatisticsGen`、`SchemaGen`、および `ExampleValidator` に使用されており、TFDV を使って、これらのコンポーネントの出力を可視化することができます。

このチュートリアルでは、TFDV を内部的に使用して可視化を表示する可視化ヘルパーメソッドを TFX に使用します。各コンポーネントの詳細については、[TFX コンポーネントのチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras)をご覧ください。

#### ExampleGen の出力を調べる

ExampleGen の出力を調べてみましょう。Split ごとに最初の 2 つの Example を見てみます。

In [ ]:
preview_examples(examples_artifacts)

デフォルトでは、TFX ExampleGen は Example を *train* と *eval* の 2 つの Split に分割しますが、[Split 構成を調整する](https://www.tensorflow.org/tfx/guide/examplegen#span_version_and_split)ことが可能です。

#### StatisticsGen の出力を調べる


In [ ]:
visualize_artifacts(stats_artifacts)

これらの統計は、データのスキーマを自動的に作成するために SchemaGen に提供されます。

#### SchemaGen の出力を調べる


In [ ]:
visualize_artifacts(schema_artifacts)

このスキーマは、StatisticsGen の出力から自動的に推論されます。このチュートリアルでは生成されたスキーマを使用しますが、[スキーマを変更してカスタマイズする](https://www.tensorflow.org/tfx/guide/statsgen#creating_a_curated_schema)ことも可能です。

#### ExampleValidator の出力を調べる


In [ ]:
visualize_artifacts(anomalies_artifacts)

異常が見つかった場合は、すべての Example が自分の想定に従っているかについて、データを調べることができます。StatistcsGen などの他のコンポーネントの出力が役立つ場合があります。見つかった異常によってパイプラインの実行がブロックされることはありません。

`SchemaGen` の出力から、利用可能な特徴量を見ることができます。特徴量が直接 `Trainer` で ML モデルを構築するために使用できる場合は、次のステップを省略してステップ 4 にお進みください。使用できない場合は、次のステップで特徴量エンジニアリングを実行できます。平均を計算するなどのフル pass 演算が必要な場合や、スケーリングが必要jな場合は特に、`Transform` コンポーネントが必要となります。

## ステップ 3.（オプション）Transform コンポーネントを使った特徴量エンジニアリング

このステップでは、パイプラインで `Transform` コンポーネントが使用するいくつかの特徴量エンジニアリングジョブを定義します。詳細については、[Transform コンポーネントガイド](https://www.tensorflow.org/tfx/guide/transform)をご覧ください。

このステップは、トレーニングコードで、ExampleGen の出力に提供されていない追加の特徴量が必要な場合にのみ必要です。そうでない場合は、Trainer の使用に関する次のステップにお進みください。

### モデルの特徴量を定義する

`models/features.py` には、特徴量名、語彙のサイズなど、モデルの特徴量を定義する定数が含まれます。`penguin` モデルは、教師あり学習モデルを使って分類の問題を解決し、すべての特徴量は連続した数値特徴量であるため、`penguin` テンプレートにはデフォルトで `FEATURE_KEYS` と `LABEL_KEY` の 2 つの定数があります。別の例については、[シカゴタクシーの例の特徴量定義](https://github.com/tensorflow/tfx/blob/master/tfx/experimental/templates/taxi/models/features.py)をご覧ください。


### preprocessing_fn() でトレーニング/サービング用の前処理を実装する

実際の特徴量エンジニアリングは、`models/preprocessing.py` の`preprocessing_fn()` 関数で行われます。

`preprocessing_fn` では、テンソルの入力ディクショナリを操作する一連の関数を定義して、テンソルの出力ディクショナリを生成することができます。TensorFlow Transform API には `scale_to_0_1` や `compute_and_apply_vocabulary` といったヘルパー関数も用意されています。または通常の TensorFlow 関数を使用することも可能です。`penguin` テンプレートにはデフォルトで、特徴量値を正規化するための [tft.scale_to_z_score](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/scale_to_z_score) 関数の使用例が含まれています。

`preprocessing_fn` のオーサリングについての詳細は、[Tensflow Transform ガイド](https://www.tensorflow.org/tfx/transform/get_started)をご覧ください。


### パイプラインに Transform コンポーネントを追加する

preprocessing_fn の準備ができたら、パイプラインに `Transform` コンポーネントを追加します。

1. `pipeline/pipeline.py` ファイルで、`# components.append(transform)` をコメント解除し、そのコンポーネントをパイプラインに追加します。

パイプラインを更新して、もう一度実行できます。

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

パイプラインが正常に実行したら、ログの*どこか*に「Component Transform is finished.」が表示されます。`Transform` コンポーネントと `ExampleValidator` コンポーネントは互いに依存していないため、実行順は不定です。とは言え、`Transform` と `ExampleValidator` のいずれかがパイプライン実行の最後のコンポーネントとなる可能性があります。

### Transform の出力を調べる

Transform コンポーネントは、Tensorflow グラフと変換済みの Example の 2 種類の出力を作成します。変換された Example は、ExampleGen からも生成される Examples アーティファクトタイプですが、これには、代わりに変換済みの特徴量値が含まれます。

これらは前のステップと同じ方法で調べることが可能です。

In [ ]:
with metadata.Metadata(metadata_connection_config) as metadata_handler:
    # Search all aritfacts from the previous run of Transform component.
    artifacts = get_latest_artifacts(metadata_handler.store,
                                     PIPELINE_NAME, "Transform")
    # Find artifacts of Examples type.
    transformed_examples_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.Examples.TYPE_NAME)

In [ ]:
preview_examples(transformed_examples_artifacts)

## ステップ 4. Trainer コンポーネントでモデルをトレーニングする

`Trainer` コンポーネントを使って ML モデルを構築します。詳細については、[Trainer コンポーネントガイド](https://www.tensorflow.org/tfx/guide/trainer)をご覧ください。Trainer コンポーネントにモデルコードを提供する必要があります。

### モデルを定義する

penguin テンプレートでは、`Trainer` コンポーネントの `run_fn` 引数として `models.model.run_fn` が使用されています。これは、`models/model.py` の `run_fn()` 関数が、`Trainer` コンポーネントが実行する際に呼び出されるということです。特定のコードで `keras` API を使用すると、単純な DNN モデルを構築するコードを確認できます。TFX での Keras API の使用についての詳細は、[TFX における TensorFlow 2.x](https://www.tensorflow.org/tfx/guide/keras)ガイドをご覧ください。

この `run_fn` では、コンポーネントによって指定される `fn_args.serving_model_dir` がポイントするディレクトリにモデルを構築して保存する必要があります。`run_fn` に渡される `fn_args` に他の引数を使用することが可能です。`fn_args` の引数の全リストについては. [関連するコード](https://github.com/tensorflow/tfx/blob/b01482442891a49a1487c67047e85ab971717b75/tfx/components/trainer/executor.py#L141)をご覧ください。

必要に応じて、`models/features.py` に特徴量を定義して使用します。ステップ 3 で特徴量を変換している場合は、モデルへの入力として変換した特徴量が確認できます。

### パイプラインに Trainer コンポーネントを追加する

run_fn の準備ができたら、パイプラインに  `Trainer` コンポーネントを追加します。

1. `pipeline/pipeline.py` ファイルで、`# components.append(trainer)` をコメント解除し、そのコンポーネントをパイプラインに追加します。

trainer コンポーネントの引数は、Transform コンポーネントを使用しているかどうかによって異なる場合があります。

- `Transform` コンポーネントを**使用していない**場合は、引数を変更する必要はありません。

- `Transform` コンポーネントを使用している場合は、`Trainer` コンポーネントのインスタンスを作成する際に、引数を変更する必要があります。

    - `examples` 引数を`examples=transform.outputs['transformed_examples'],` に変更します。トレーニングには変換済みの Example を使用する必要があります。
    - `transform_graph=transform.outputs['transform_graph'],` のように、`transform_graph` を追加します。このグラフには、変換演算の TensorFlow グラフが含まれます。
    - 上記の変更を行ったら、Trainer コンポーネント作成のコードは以下のようになります。

    ```python
    # If you use a Transform component.
    trainer = Trainer(
        run_fn=run_fn,
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        ...
    ```

パイプラインを更新して、もう一度実行できます。

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

この実行が正常に行われると、自分のモデルに使用する初めての TFX パイプラインの作成が完了です。お疲れ様でした！

新しいモデルは、出力ディレクトリ配下のどこかにありますが、中間結果が多数含まれる TFX パイプライン外部の一定した場所かサービスにモデルを維持するのがよいでしょう。構築したしたモデルを継続的に評価するとさらによいでしょう。ML 本番システムにおいては非常に重要なことです。次のステップでは、継続的評価とデプロイの仕組みを確認しましょう。

## ステップ 5.（オプション）Evaluator でモデルを評価し、Pusher で公開する


[`Evaluator`](https://www.tensorflow.org/tfx/guide/evaluator) コンポーネントは、`Trainer` から構築された各モデルを継続的に評価し、[`Pusher`](https://www.tensorflow.org/tfx/guide/pusher) はそのモデルをファイルシステムの事前定義の場所や [Google Cloud AI Platform Models](https://console.cloud.google.com/ai-platform/models) にコピーします。

### パイプラインに Evaluator コンポーネントを追加する

`pipeline/pipeline.py` ファイル:

1. `# components.append(model_resolver)` をコメント解除し、最新のモデルレゾルバ―をパイプラインに追加します。Evaluator は、最後のパイプラインランで Evaluator に渡された古い基準モデルに対し、モデルを比較するために使用することができます。Evaluator に渡された最後のモデルは、`LatestBlessedModelResolver` が見つけます。
2. モデルに適切な `tfma.MetricsSpec` を設定します。評価は、ML モデルごとに異なる場合があります。penguin テンプレートでは、複数のカテゴリの分類問題を解決しているため、`SparseCategoricalAccuracy` が使用されていました。特定のスライスについてモデルを分析するには、`tfma.SliceSpec` を指定する必要もあります。詳細については、[Evaluator コンポーネントガイド](https://www.tensorflow.org/tfx/guide/evaluator)をご覧ください。
3. `# components.append(evaluator)` をコメント解除し、そのコンポーネントをパイプラインに追加します。

パイプラインを更新して、もう一度実行できます。

In [ ]:
# Update and run the pipeline.
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

### Evaluator の出力を調べる

このステップでは、TensorFlow Model Analysis（TFMA）Jupyter Notebook 拡張機能が必要です。TEMA Notebook 拡張機能のバージョンは、TFMA Python パッケージと同じである必要があります。

以下のコマンドは NPM レジストリから TFMA Notebook 拡張機能をインストールします。完了までに数分かかる場合があります。

In [ ]:
# Install TFMA notebook extension.
!jupyter labextension install tensorflow_model_analysis@{tfma.__version__}

インストールが完了したら、**ブラウザを再読み込み**して、拡張機能を有効にしてください。

In [ ]:
with metadata.Metadata(metadata_connection_config) as metadata_handler:
  # Search all aritfacts from the previous pipeline run.
  artifacts = get_latest_artifacts(metadata_handler.store, PIPELINE_NAME)
  model_evaluation_artifacts = find_latest_artifacts_by_type(
      metadata_handler.store, artifacts,
      standard_artifacts.ModelEvaluation.TYPE_NAME)

In [ ]:
if model_evaluation_artifacts:
  tfma_result = tfma.load_eval_result(model_evaluation_artifacts[0].uri)
  tfma.view.render_slicing_metrics(tfma_result)

### パイプラインに Pusher コンポーネントを追加する

モデルが有望だと思える場合は、モデルを公開する必要があります。[Pusher コンポーネント](https://www.tensorflow.org/tfx/guide/pusher)は、ファイルシステムの場所または[カスタム Executor](https://github.com/tensorflow/tfx/blob/master/tfx/extensions/google_cloud_ai_platform/pusher/executor.py) の使用により GCP AI Platform Models にモデルを公開できます。

`Evaluator` コンポーネントは、`Trainer` から構築された各モデルを継続的に評価し、[`Pusher`](https://www.tensorflow.org/tfx/guide/pusher) はそのモデルをファイルシステムの事前定義の場所や [Google Cloud AI Platform Models](https://console.cloud.google.com/ai-platform/models) にコピーします。

1. `local_runner.py` で、`SERVING_MODEL_DIR` を、公開するディレクトリに設定します。
2. `pipeline/pipeline.py` ファイルで、`# components.append(pusher)` をコメント解除し、Pusher をパイプラインに追加します。

パイプラインを更新して、もう一度実行できます。

In [ ]:
# Update and run the pipeline.
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

`SERVING_MODEL_DIR` に新しいモデルを見つけられます。

## ステップ 6.（オプション）GCP の Kubeflow Pipelines にパイプラインをデプロイする


前述のとおり、`local_runner.py` はデバッグや開発の目的に適していますが、本番のワークロードにおいては最適なソリューションとは言えません。このステップでは、Google Cloud の Kubeflow Pipelines にパイプラインをデプロイします。

### 準備

パイプラインを Kubeflow Pipelines クラスタにデプロイするには、`kfp` python パッケージと `skaffold` プログラムが必要です。

In [ ]:
!pip install --upgrade -q kfp

# Download skaffold and set it executable.
!curl -Lo skaffold https://storage.googleapis.com/skaffold/releases/latest/skaffold-linux-amd64 && chmod +x skaffold

`skaffold` バイナリを、シェルが見つけられる場所に移動する必要があります。または、`tfx` バイナリを `--skaffold-cmd` フラグで実行する際には skaffold へのパスを指定することができます。

In [ ]:
# Move skaffold binary into your path
!mv skaffold /home/jupyter/.local/bin/

パイプラインを実行するための Kubeflow Pipelines クラスタも必要です。[Cloud AI Platform Pipelines での TFX のチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/cloud-ai-platform-pipelines)に記載されているステップ 1 と 2 を実行してください。

クラスタの準備ができたら、[Google cloud console の `Pipelines` ページ](http://console.cloud.google.com/ai-platform/pipelines)で、*Open Pipelines Dashboard（パイプラインダッシュボードを開く）*をクリックし、パイプラインダッシュボードを開きます。このページの URL は、パイプラインランをリクエストする `ENDPOINT` です。エンドポイント値は、https:// の後から googleusercontent.com までです。以下のコードブロックに自分のエンドポイントを入力してください。


In [ ]:
ENDPOINT='' # Enter your ENDPOINT here.

Kubeflow Pipelines クラスタでコードを実行するには、コードをコンテナイメージにパックする必要があります。このイメージjは、パイプラインをデプロイするときに自動的に構築されるため、イメージの名前とコンテナレジストリを設定することだけが必要となります。この例では、[Google Container Registry](https://cloud.google.com/container-registry) を使用して、`tfx-pipeline` と名付けます。

In [ ]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT=shell_output[0]

# Docker image name for the pipeline image.
CUSTOM_TFX_IMAGE='gcr.io/' + GOOGLE_CLOUD_PROJECT + '/tfx-pipeline'

### データの場所を設定する

データは Kubeflow Pipelines クラスタからアクセスできる必要があります。ローカル環境のデータを使用した場合は、Google Cloud Storage などのリモートストレージにアップロードする必要があるかもしれません。たとえば、ペンギンのデータを、Kubeflow Pipelines クラスタがデプロイされたときに自動的に作成されるデフォルトのバケットにアップロードできます。

In [ ]:
!gsutil cp data/data.csv gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/tfx-template/data/penguin/

`kubeflow_runner.py` の `DATA_PATH` に保存されたデータ場所を更新します。

BigQueryExampleGen を使用している場合は、データファイルをアップロードする必要はありませんが、`kubeflow_runner.py` が `pipeline.create_pipeline()` 関数と同じ `query` と `beam_pipeline_args` 引数を使用することを確認してください。

### パイプラインをデプロイする

すべての準備が整ったら、`tfx pipeline create` コマンドを使ってパイプラインを作成できます。

> 注意: Kubeflow Pipelines のパイプラインを作成するときは、パイプラインの実行に使用されるコンテナイメージが必要です。そして、`skaffold` がイメージを構築します。`skaffold` は Docker ハブからベースイメージをプルするため、最初にイメージをビルドするときは 5〜10 分かかりますが、2 回目以降のビルドにはそれほど時間がかかりません。


In [ ]:
!tfx pipeline create  \
--engine=kubeflow \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT} \
--build-target-image={CUSTOM_TFX_IMAGE}

次に、`tfx run create` コマンドを使用して、新しく作成されたパイプラインで実行を開始します。

In [ ]:
!tfx run create --engine=kubeflow --pipeline-name={PIPELINE_NAME} --endpoint={ENDPOINT}

または、Kubeflow Pipelines ダッシュボードでパイプラインを実行することもできます。新しいランは、Kubeflow Pipelines ダッシュボードの `Experiments` の下に一覧表示されます。実験をクリックすると、進行状況を監視し、実行中に作成されたアーティファクトを可視化できます。

Kubeflow Pipelines でパイプラインを実行することに興味がある場合は、詳細な指示について、[Cloud AI Platform Pipelines での TFX のチュートリアル](https://www.tensorflow.org/tfx/tutorials/tfx/cloud-ai-platform-pipelines)をご覧ください。

### クリーンアップ

このステップで使用されているすべての Google Cloud リソースをクリーンアップするには、チュートリアルで使用した [Google Cloud プロジェクトを削除](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)できます。

または、各コンソールにアクセスして、個々のリソースをクリーンアップすることもできます。

- [Google Cloud Storage](https://console.cloud.google.com/storage)
- [Google Container Registry](https://console.cloud.google.com/gcr)
- [Google Kubernetes Engine](https://console.cloud.google.com/kubernetes)